In [56]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

import datetime as dt
import plotly.graph_objects as go

from tqdm import tqdm

In [2]:
data = pd.read_parquet("stock_data.parquet")
stock_names = data['comnam'].unique()

# repeat = True
# while repeat:
#     count = 0
#     dfs = []
#     sizes = []
#     relevant_stocks = np.random.choice(stock_names,size=10, replace=False)
#     for stock in relevant_stocks:
#         subset = data[data['comnam'] == stock]
#         df = subset[['prc']]
#         df.columns = [stock]
#         df.index = pd.to_datetime(subset['date'])
#         dfs.append(df)
#         sizes.append(df.shape[0])
#     dfs = [x for _,x in reversed(sorted(zip(sizes,dfs), key = lambda x:x[0]))]
#     final = dfs[0]
#     for i in range(1, 10):
#         final = final.join(dfs[i])
#     final.index.name = 'date'
#     final = final.groupby(by='date').mean()
#     final = final.interpolate()
#     period = np.random.randint(30, final.shape[0])
#     final = final.iloc[period-30:period]
#     if final.iloc[0].isna().any() or final.iloc[29].isna().any():
#         repeat = True
#     else:
#         repeat = False

In [182]:
data

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,bid,ask,shrout,cfacpr,cfacshr,openprc,numtrd,retx,ticker,comnam
33229,68389X10,10104.0,8045.0,10536.0,1.0,7372.0,2020-01-02,53.230000,53.959999,53.950001,...,53.950001,53.959999,3207649.0,1.0,1.0,53.270000,NaN,0.018309,ORCL,ORACLE SYSTEMS CORP
33230,68389X10,10104.0,8045.0,10536.0,1.0,7372.0,2020-01-03,52.950001,54.049999,53.759998,...,53.770000,53.779999,3207649.0,1.0,1.0,52.990002,NaN,-0.003522,ORCL,ORACLE SYSTEMS CORP
33231,68389X10,10104.0,8045.0,10536.0,1.0,7372.0,2020-01-06,53.349998,54.200001,54.040001,...,54.040001,54.049999,3207649.0,1.0,1.0,53.360001,NaN,0.005208,ORCL,ORACLE SYSTEMS CORP
33232,68389X10,10104.0,8045.0,10536.0,1.0,7372.0,2020-01-07,53.610001,54.330002,54.160000,...,54.160000,54.169998,3207649.0,1.0,1.0,53.889999,NaN,0.002221,ORCL,ORACLE SYSTEMS CORP
33233,68389X10,10104.0,8045.0,10536.0,1.0,7372.0,2020-01-08,53.700001,54.599998,54.130001,...,54.119999,54.130001,3207649.0,1.0,1.0,53.939999,NaN,-0.000554,ORCL,ORACLE SYSTEMS CORP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340541,88160R10,93436.0,53453.0,66252.0,3.0,9999.0,2022-03-25,997.320129,1021.799927,1010.640015,...,1010.210022,1010.890015,1033534.0,1.0,1.0,1008.000000,593068.0,-0.003235,TSLA,TESLA INC
340542,88160R10,93436.0,53453.0,66252.0,3.0,9999.0,2022-03-28,1053.599976,1097.879883,1091.839966,...,1091.839966,1092.199951,1033534.0,1.0,1.0,1065.099976,1113132.0,0.080345,TSLA,TESLA INC
340543,88160R10,93436.0,53453.0,66252.0,3.0,9999.0,2022-03-29,1073.109985,1114.770020,1099.569946,...,1099.819946,1099.959961,1033534.0,1.0,1.0,1107.989990,835619.0,0.007080,TSLA,TESLA INC
340544,88160R10,93436.0,53453.0,66252.0,3.0,9999.0,2022-03-30,1084.000000,1113.949951,1093.989990,...,1093.689941,1093.989990,1033534.0,1.0,1.0,1091.170044,598381.0,-0.005075,TSLA,TESLA INC


In [188]:
# Pivot data (columns: stock name, index: date)
data_pivot = data.pivot_table(index="date", values="prc", columns="comnam")

# Remove stocks with na in price column random stocks
# TODO: Remove the complete stock if any na
data_clean = data_pivot.dropna(axis=1, how='any')
data_clean
# # Pick 10 stocks at random
# relevant_stocks = np.random.choice(data_clean.columns,size=10, replace=False)
# data_filt = data_clean[relevant_stocks]

# # Focus on window of 30 trading days with random start dt
# start_dt = np.random.choice(data_filt.index[:-30],size=1, replace=False)[0]
# final = data_pivot.query("date >= @start_dt")
# final = final.iloc[:30]

# final
# data_filt

comnam,10X GENOMICS INC,3M CO,A C C O BRANDS CORP,A D T INC,A E S CORP,A H BELO CORP,A M B PROPERTY CORP,A M N HEALTHCARE SERVICES INC,A M R CORP DEL,A P A CORP,...,ZAYRE CORP,ZEBRA TECHNOLOGIES CORP,ZILLOW GROUP INC,ZILLOW INC,ZIMMER BIOMET HOLDINGS INC,ZIMMER HOLDINGS INC,ZIONS BANCORPORATION,ZIONS BANCORPORATION N A,ZIONS UTAH BANCORPORATION,ZOETIS INC
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,72.830002,180.000000,9.11,7.57,20.030001,2.830,88.400002,62.110001,29.090000,25.360001,...,61.700001,259.140015,45.157499,45.000000,149.199997,149.199997,52.439999,52.439999,52.439999,134.139999
2020-01-03,75.559998,178.449997,8.99,7.45,19.799999,2.820,89.389999,62.419998,27.650000,25.690001,...,61.240002,256.049988,44.680001,44.560001,148.809998,148.809998,51.700001,51.700001,51.700001,134.160004
2020-01-06,75.550003,178.619995,9.08,7.56,20.030001,2.920,89.040001,62.110001,27.320000,25.639999,...,61.630001,258.010010,44.587500,44.430000,147.949997,147.949997,51.070000,51.070000,51.070000,133.130005
2020-01-07,75.980003,177.899994,8.87,7.44,20.100000,2.910,88.470001,62.049999,27.219999,32.509998,...,61.590000,256.470001,45.697500,45.480000,147.820007,147.820007,50.759998,50.759998,50.759998,133.580002
2020-01-08,74.839996,180.630005,8.84,6.81,20.059999,2.891,88.949997,63.490002,27.840000,32.730000,...,61.919998,247.639999,45.672501,45.380001,149.589996,149.589996,51.259998,51.259998,51.259998,133.289993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,67.790001,150.460007,8.19,7.36,24.370001,6.940,157.330002,102.250000,17.010000,41.529999,...,61.290001,425.660004,51.742500,50.849998,125.279999,125.279999,71.080002,71.080002,71.080002,189.429993
2022-03-28,70.410004,149.470001,8.19,7.45,24.650000,7.030,160.350006,105.510002,17.299999,39.849998,...,62.000000,422.279999,52.474999,51.500000,125.680000,125.680000,69.730003,69.730003,69.730003,189.369995
2022-03-29,77.250000,152.080002,8.34,7.85,25.450001,6.970,164.699997,106.440002,18.160000,40.959999,...,64.309998,438.100006,55.107499,54.110001,129.580002,129.580002,69.739998,69.739998,69.739998,192.279999


Create an initial portfolio

In [187]:
def get_prices_window(prices, start_dt):
    # Pick 10 stocks at random
    relevant_stocks = np.random.choice(prices.columns,size=10, replace=False)
    prices_filt = prices[relevant_stocks]

    final = prices_filt.query("date >= @start_dt")
    final = final.iloc[:30]
    return final

def generate_portfolios(stock_prices, init_budget=100000):
    initial_budget = init_budget
    initial_prices = stock_prices.iloc[0].to_numpy()
    # available_money = initial_budget * .95
    available_money = initial_budget
    P_0 = np.zeros(10) #initial portfolio
    break_bool = True
    ignore =  np.random.randint(0,10)
    while break_bool:
        stock_buy = np.random.randint(0,10)
        price = initial_prices[stock_buy]
        if stock_buy != ignore:
            if (available_money * .75) - price >= 0:
                available_money = available_money - price
                P_0[stock_buy] = P_0[stock_buy] + 1
            else:
                break_bool = False

    # w_0 = initial_budget * .05 + available_money
    w_0 = available_money
    # print(w_0)
    # print(P_0)
    final_prices = stock_prices.iloc[29].to_numpy()
    available_money = initial_budget * .9
    break_bool = True
    P_t = np.zeros(10) #target portfolio

    while break_bool:
        stock_buy = np.random.randint(0,10)
        price = final_prices[stock_buy]
        if available_money - price >= 0:
            available_money = available_money - price
            P_t[stock_buy] = P_t[stock_buy] + 1
        else:
            break_bool = False
    # print(P_t)
    return P_0, P_t, w_0, initial_prices, final_prices

# Formulations

## With Perfect Information

In [277]:
def get_naive_opt(P_0, P_t, w_0, initial_budget, initial_prices, final_prices, print_bool=False):
    F = 5.95 * np.ones(10)# trading fees
    # P_0 = starting portfolio
    # P_t = target portfolio
    # w_0 = starting cash
    # w_f = final free cash
    # budget = 20000
    M = initial_budget

    # Silence Gurobi output
    env = gp.Env(empty=True)
    env.setParam("OutputFlag",0)
    env.start()

    m = gp.Model(env=env)
    # Define Variables
    P_buy = m.addMVar((10,), vtype=GRB.INTEGER)
    P_sell = m.addMVar((10,), vtype=GRB.INTEGER)
    z_sell = m.addMVar((10,), vtype=GRB.BINARY)
    z_buy = m.addMVar((10,), vtype=GRB.BINARY)
    # Constraints

    ## Meet Target
    m.addConstr(
        P_0 + P_buy - P_sell >= P_t
    )

    ## Have Money
    m.addConstr(
        w_0 + initial_prices @ P_sell - initial_prices @ P_buy - F @ z_sell - F @ z_buy >=0
    )

    ## if we sell a stock, we pay a trading price
    m.addConstr(
        M*(z_sell) >= P_sell
    )

    ## If we buy a stock, we pay a trading fee

    m.addConstr(
        M*(z_buy) >= P_buy
    )

    cash = w_0 + initial_prices @ P_sell - initial_prices @ P_buy
    # Set objective: Maximize end portfolio value
    m.ModelSense = GRB.MAXIMIZE
    # try:
    m.setObjective(
        # cash - F @ z_sell - F @ z_buy + final_prices @ (P_0 + P_buy - P_sell)
        cash + final_prices @ (P_0 + P_buy - P_sell)
    )
    # except Exception as e:
        # print(e)
        # print((cash - F @ z_sell - F @ z_buy + final_prices @ (P_0 + P_buy - P_sell)).getValue())
        # print(cash.x)
        # print(z_sell.x)
        # print(z_buy.x)
        # print(final_prices)
        # print(P_0.x)
        # print(P_buy.x)
        # print(P_sell.x)

    m.optimize()
    final_portfolio = P_0 + P_buy.x - P_sell.x
    # free_cash = w_0 + initial_prices @ P_sell.x - initial_prices @ P_buy.x
    cash = cash.getValue()[0]
    final_portfolio_value = cash + final_prices @ final_portfolio
    
    if print_bool:
        print(f"Init Portf Value: {initial_prices @ P_0}")
        print(f"Final Portfolio Value: {final_portfolio_value}")
        print(f"Cash Init: {w_0[0]}")
        print(f"Cash Final: {cash}")
        print(f"Init Portfolio: {P_0}")
        print(f"Final Portfolio: {final_portfolio}")
        print(f"Target Portfolio: {P_t}")
        print(f"TX Vec Buy: {z_buy.x}")
        print(f"TX Vec Sell: {z_sell.x}")
        print(f"Total Tx Costs: {F @ z_sell.x + F @ z_buy.x}")
    return final_portfolio_value, final_portfolio, cash

In [278]:
# test
rng = np.random.default_rng()
budget = rng.integers(low=1000, high=10000, size=1)
start_dt = np.random.choice(data_pivot.index[:-30], size=1, replace=True)[0]
stock_price_window = get_prices_window(data_clean, start_dt)
P_0, P_t, w_0, initial_prices, final_prices = generate_portfolios(stock_prices = stock_price_window, init_budget=budget)
final_portfolio_value, final_portfolio, free_cash = get_naive_opt(P_0, P_t, w_0, budget, initial_prices, final_prices, print_bool=True)

Init Portf Value: 7537.364938735962
Final Portfolio Value: 8105.750235557556
Cash Init: 410.6350612640381
Cash Final: 65.62504005432129
Init Portfolio: [8. 9. 7. 0. 6. 6. 8. 4. 3. 6.]
Final Portfolio: [ 7.  5.  8.  8.  6.  4.  5.  9. 44.  7.]
Target Portfolio: [7. 5. 8. 8. 6. 4. 5. 9. 7. 7.]
TX Vec Buy: [-0.  0.  1.  1.  0.  0.  0.  1.  1.  1.]
TX Vec Sell: [1. 1. 0. 0. 0. 1. 1. 0. 0. 0.]
Total Tx Costs: 53.55


## Multi-stage

##

In [274]:
def get_multi_stage_opt(P_0, P_t, w_0, initial_budget, stock_price_window, print_bool=False):   
    # Silence Gurobi output
    env = gp.Env(empty=True)
    env.setParam("OutputFlag",0)
    env.start()

    F = 5.95  * np.ones(10)# trading fees
    # prices = stock_price_window.interpolate().to_numpy()
    prices = stock_price_window.to_numpy()
    M = initial_budget
    # P_0 = starting portfolio
    # P_t = target portfolio
    # w_0 = starting cash

    m = gp.Model(env=env)

    # Define Variables
    P_buy = m.addMVar((30,10), vtype=GRB.INTEGER)
    P_sell = m.addMVar((30,10), vtype=GRB.INTEGER)
    z_sell = m.addMVar((30,10), vtype=GRB.BINARY)
    z_buy = m.addMVar((30,10), vtype=GRB.BINARY)
    P_time = m.addMVar((30,10), vtype=GRB.INTEGER)
    # P_time = np.ndarray((30,10))
    w = m.addMVar((30,), vtype=GRB.CONTINUOUS, lb = 0)
    # w = np.ndarray((30,))
    # Constraints

    ## Represent Portfolios across time

    ### Time step 0
    m.addConstr(
        P_time[0,:] == P_0 + P_buy[0,:] - P_sell[0,:]
    )
    ### Rest of the time
    m.addConstrs((
        P_time[i,:] == P_time[i-1,:] - P_buy[i,:] - P_sell[i,:] for i in range(1,30)
    ))

    ### Final_target
    m.addConstr(
        P_time[29,:] >= P_t
    )

    ## Represent money
    ### First time step
    m.addConstr(
        w[0] == w_0 - prices[0,:] @ P_buy[0,:] + prices[0,:] @ P_sell[0,:] - F @ z_buy[0,:] - F @ z_sell[0,:]
    )
    ## Rest of the time
    m.addConstrs((
        w[i] == w[i-1] - prices[i,:] @ P_buy[i,:] + prices[i,:] @ P_sell[i,:] - F @ z_buy[i,:] - F @ z_sell[i,:] for i in range(1,30)
    ))

    ## Trading fees
    ### if we sell a stock, we pay a trading price
    m.addConstr(
        M*(z_sell) >= P_sell
    )

    ### If we buy a stock, we pay a trading fee
    m.addConstr(
        M*(z_buy) >= P_buy
    )

    # Maximize end Portfolio value
    m.ModelSense = GRB.MAXIMIZE
    m.setObjective(
        w[29] + prices[29,:] @ P_time[29,:]
    )
    m.optimize()
    final_portfolio = P_time[29,:].x
    cash = w[29].x 
    final_portfolio_value = cash + prices[29,:] @ final_portfolio
    # print(f"Final Portfolio Value: {final_portfolio_value}")
    # print(f"Free Cash: {free_cash}")
    # print(f"Final Portfolio: {final_portfolio}")
    if print_bool:
        print(f"Init Portf Value: {initial_prices @ P_0}")
        print(f"Final Portfolio Value: {final_portfolio_value}")
        print(f"Cash Init: {w_0[0]}")
        print(f"Cash Final: {cash}")
        print(f"Init Portfolio: {P_0}")
        print(f"Final Portfolio: {final_portfolio}")
        print(f"Target Portfolio: {P_t}")
        print(f"TX Vec Buy: {z_buy.x}")
        print(f"TX Vec Sell: {z_sell.x}")
        print(f"Total Tx Costs: {[F @ z_sell.x[i] + F @ z_buy.x[i] for i in range(30)]}")
    return final_portfolio_value, final_portfolio, free_cash

In [275]:
# test
rng = np.random.default_rng()
budget = rng.integers(low=1000, high=10000, size=1)
start_dt = np.random.choice(data_pivot.index[:-30], size=1, replace=True)[0]
stock_price_window = get_prices_window(data_clean, start_dt)
P_0, P_t, w_0, initial_prices, final_prices = generate_portfolios(stock_prices = stock_price_window, init_budget=budget)
final_portfolio_value, final_portfolio, free_cash = get_multi_stage_opt(P_0, P_t, w_0, budget, stock_price_window, print_bool=True)

Init Portf Value: 5778.230003356934
Final Portfolio Value: 5492.6100051879885
Cash Init: 325.7699966430664
Cash Final: 240.98000411987312
Init Portfolio: [ 0.  3.  4.  7.  8.  2.  9.  3. 10.  8.]
Final Portfolio: [5. 4. 3. 4. 7. 4. 7. 6. 4. 4.]
Target Portfolio: [5. 4. 3. 4. 7. 4. 7. 6. 4. 4.]
TX Vec Buy: [[ 1.  1. -0.  0. -0.  1.  0.  1. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0.

In [279]:
simulations = 10
rng = np.random.default_rng()
budget_vec = rng.integers(low=1000, high=10000, size=simulations)
start_dt_vec = np.random.choice(data_pivot.index[:-30], size=simulations, replace=True)
col_names = ["portf_val", "final_portf", "cash", "init_budget"]
# df_naive = pd.DataFrame(columns = )
# df_multi = pd.DataFrame(columns = ["portf_val", "final_portf", "cash"])
naive_list = list()
multi_list = list()

# Could parallelize this but fuck it
for i, budget in enumerate(tqdm(budget_vec)):
    # try:
        stock_price_window = get_prices_window(data_clean, start_dt_vec[i])
        P_0, P_t, w_0, initial_prices, final_prices = generate_portfolios(stock_prices = stock_price_window, init_budget=budget)
        final_portfolio_value, final_portfolio, free_cash = get_naive_opt(P_0, P_t, w_0, budget, initial_prices, final_prices, print_bool=True)
        naive_list.append(pd.Series([final_portfolio_value, final_portfolio, free_cash, budget], index = col_names))
        final_portfolio_value, final_portfolio, free_cash = get_multi_stage_opt(P_0, P_t, w_0, budget, stock_price_window)
        multi_list.append(pd.Series([final_portfolio_value, final_portfolio, free_cash, budget], index = col_names))
    # except Exception as e:
        # pass
        # print(e)

df_naive = pd.concat(naive_list, axis=1).T
df_multi = pd.concat(multi_list, axis=1).T

  0%|          | 0/10 [00:00<?, ?it/s]


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [245]:
df_naive

,portf_val,final_portf,cash,init_budget
0,4468.699937,"[4.0, 7.0, 6.0, 7.0, 4.0, 6.0, 4.0, 2.0, 15.0,...",0.069966,4327
1,9959.020044,"[3.0, 12.0, 14.0, 14.0, 14.0, 9.0, 8.0, 6.0, 6...",1.169928,9466
2,5861.309903,"[11.0, 5.0, 4.0, 5.0, 6.0, 10.0, 5.0, 5.0, 5.0...",2.459898,5159
3,7264.189882,"[7.0, 7.0, 8.0, 5.0, 1.0, 4.0, 6.0, 7.0, 5.0, ...",0.810032,6563
4,1494.21004,"[1.0, 1.0, 0.0, 4.0, 1.0, 0.0, 4.0, 0.0, 1.0, ...",0.749995,1406
...,...,...,...,...
77,8926.639915,"[4.0, 9.0, 9.0, 5.0, 12.0, 8.0, 10.0, 5.0, 10....",3.930078,7932
78,7130.180113,"[3.0, 4.0, 5.0, 3.0, 6.0, 6.0, 8.0, 4.0, 41.0,...",2.570024,6371
79,9662.15004,"[11.0, 10.0, 7.0, 7.0, 4.0, 9.0, 4.0, 9.0, 4.0...",7.849926,9017
80,1840.059961,"[0.0, 2.0, 2.0, 0.0, 1.0, 4.0, 2.0, 0.0, 1.0, ...",11.139996,1640


In [73]:
df_multi

,portf_val,final_portf,cash,init_budget
0,4574.379947,"[2.0, 7.0, 6.0, 7.0, 4.0, 6.0, 4.0, 2.0, 11.0,...",683.5099859237671,4327
1,10242.340117,"[3.0, 12.0, 14.0, 13.0, 14.0, 9.0, 8.0, 6.0, 1...",1848.1799945831299,9466
2,6074.169959,"[6.0, 5.0, 4.0, 4.0, 6.0, 10.0, 5.0, 5.0, 5.0,...",1475.6499633789062,5159
3,7291.049894,"[7.0, 7.0, 8.0, 5.0, 1.0, 4.0, 6.0, 7.0, 5.0, ...",1591.8699836730957,6563
4,1503.330031,"[1.0, 1.0, 0.0, 4.0, 1.0, 0.0, 4.0, 0.0, 1.0, ...",316.1500024795532,1406
...,...,...,...,...
77,9143.99993,"[4.0, 9.0, 9.0, 5.0, 12.0, 8.0, 10.0, 5.0, 10....",2028.8100061770047,7932
78,7134.189951,"[3.0, 4.0, 5.0, 3.0, 6.0, 6.0, 8.0, 4.0, 10.0,...",1263.9399089813232,6371
79,9662.15004,"[11.0, 10.0, 7.0, 7.0, 4.0, 9.0, 4.0, 9.0, 4.0...",7.849926039228432,9017
80,1840.059961,"[0.0, 2.0, 2.0, 0.0, 1.0, 4.0, 2.0, 0.0, 1.0, ...",11.139995648237772,1640


In [74]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        y = df_multi.portf_val - df_naive.portf_val,
        x = df_multi.init_budget,
        mode = "markers",

    )
)

fig.update_xaxes(
    title="Starting Budget"
)

fig.update_yaxes(
    title="Multi-period Advantage"
)